# <b> SHA-256 Password Cracking Algorithms</b>
Cameron Rosenberger <br>
CS 3339 <br>
March 23, 2023

### <b>Dictionary References</b>

> #####  Crack Station - 15 GB dictionary
>> -crackstation.txt<br>
>> -realhuman_phill.txt<br>
>https://crackstation.net/files/crackstation.txt.gz<br>
>https://crackstation.net/files/crackstation-human-only.txt.gz<br>
> ##### Openwall
>> -lower.txt<br>
>> -mixed.txt <br>
>> -acronym.txt<br>
>> -password.txt<br>
>> -lower_pwd.txt<br>
> https://download.openwall.net/pub/wordlists/




In [27]:
#Read in codes to be cracked
#Read in salt value
codes = []
prefix = "./"
with open("./Project Files/hashes.txt","r") as f:
    for line in f:
        codes.append(line.strip())
with open("./Project Files/salt.txt", "r") as f:
    salt = f.read().strip()
print("Salt:            ",salt)
print("First code read: ",codes[0])

Salt:             aB6nkeF0He3imq4AOhbO5kEljbveRpLn
First code read:  853cb5e6ee00f1cb4912c89e3dbb9f3d815e9aa574f794422602ec0057386e5a


In [28]:
#Create dictionaries, one small and one large
mb = []       #mixed - named from one of the file names
baf = []      #big ass file

file_names = ["lower.txt","mixed.txt","acronym.txt","password.txt","lower_pwd.txt"]
baf_file = "realhuman_phill.txt"
for file in file_names:
    path ="./Dictionaries/"+file
    with open(path,"r") as f:
        for line in f:
            mb.append(line.strip())
            
with open(prefix+"Dictionaries/"+baf_file,"r",errors='ignore') as f:
    for line in f:
        baf.append(line.strip())
print("Possible passwords: ",format(len(baf)+len(mb),',d'))

Possible passwords:  64,522,459


In [29]:
#Create lists to hold cracked passwords
cracked = []
uncracked = []

In [30]:
import hashlib
def hash_password(pwrd):
    #Generates hash of original password
    salted_password = (pwrd).encode('utf-8')
    # hash salted password
    hash_value = hashlib.sha256(salted_password).hexdigest()
    return hash_value

In [31]:
from timeit import default_timer as timer
print("CRACKING.......")
start = timer()
for code in codes:
    found = False
    for val in mb:
        guess = hash_password(salt+val)
        if guess == code:
            print(val)
            found = True
            cracked.append(code+": "+val)
            break
    if found == False:
        print("--")
        uncracked.append(code)
print("***FINISHED***")
end = timer()
print(f"Process took {round(end-start,2)} seconds")

CRACKING.......
password
--
opo
--
qwer
asdf
--
--
--
--
abc123
--
mission
admin
--
--
--
bob
alice
nsa
--
--
l33
--
--
--
--
charliebrown
--
cxja
ew8a
--
--
--
derek
123456789
111111
simpsons
bigmac
catch22
171717
--
--
--
--
--
--
--
--
--
--
tj6j
--
--
--
***FINISHED***
Process took 29.31 seconds


In [32]:
print("Cracked:   ",len(cracked))
print("Uncracked: ",len(uncracked))

Cracked:    22
Uncracked:  33


In [33]:
#same process but with much bigger password dictionary
rem = []
print("CRACKING.......")
start = timer()
for code in uncracked:
    found = False
    for val in baf:
        guess = hash_password(salt+val)
        if guess == code:
            print(val)
            found = True
            cracked.append(code+": "+val)
            break
    if found == False:
        print("--")
        rem.append(code)

print("**FINISHED**")
end = timer()
print(f"Process took {round(end-start,2)} seconds")

CRACKING.......
p@ssw0rd
potato
thiv
Painting
magazine
footballgame
Queen
--
--
password123
DRAGON
dogpark
--
penultimate
3339
d1ct10n@ry
antiestablishmentarianism
gooddog
Dune
scrabble
grendel
--
ghka
u
np
pea
aes
--
Traveler
supreme
greengrass
snow82
goodbye
**FINISHED**
Process took 1355.03 seconds


In [34]:
print("Cracked:   ",len(cracked))
print("Uncracked: ",len(rem))

Cracked:    50
Uncracked:  5


In [35]:
#Super massive dictionary, 15 GB
#Can't be used as an online source, must be downloaded
#This file is left out of the downloaded zip file becauseof its size
#For demonstration of this section, I can run it locally

fin = []
start = timer()
print("CRACKING.......")
with open("./Dictionaries/crackstation.txt","r", errors="ignore") as f:
    for code in rem:
        found = False
        for line in f:
            val = line.strip()
            guess = hash_password(salt+val)
            if guess == code:
                print(val)
                found = True
                cracked.append(code+": "+val)
                break
        if found == False:
            print("--")
            fin.append(code)
print("**FINISHED**")
end = timer()
elapsed = end-start

print(f"Process took {int(elapsed/60)}.{int(elapsed%60,)} min")

CRACKING.......
@dm1n
--
--
--
--
**FINISHED**
Process took 24.20 min


In [36]:
print("Cracked:   ",len(cracked))
print("Uncracked: ",len(fin))

Cracked:    51
Uncracked:  4


In [37]:
#Brute Force Algorithm
def make_next_guess(guess):
    carry = 1
    next_guess = guess

    for i in range(len(guess)):
        cur_char = ord(guess[i]) + carry
        if cur_char > ord('z'):
            cur_char = ord('A')
            carry = 1
        else:
            carry = 0
        next_guess = next_guess[:i] + chr(cur_char) + guess[i + 1:]
        if carry == 0:
             break
    if carry == 1:
        next_guess += 'A'

    return next_guess

In [ ]:
guess = 'A'
print("CRACKING......."+'\n')
for code in fin:
    start = timer()  
    print("Decoding: ",code)
    for i in range(58 ** 6): #password maximum length 6 and there are 58 characters that can be used
        found = False
        if hash_password(salt+guess) == code:
            cracked.append(code+": "+guess)
            print(guess)
            found = True
            break
        guess = make_next_guess(guess)
        end = timer()
        elapsed = end-start
        if elapsed > 7200: #2hrs
            print(f"Process took too long({int(elapsed/60)}.{int(elapsed%60)} min)...\nMoving on to next code")
            break
    if found == False:
        print("--")

CRACKING.......

Decoding:  26d95d4fe16b19533e1684192d212df6892b5cf4b73693abbc1df2e232782f18
Process took too long(120.0 min).../nMoving on to next code
--
Decoding:  bf17e8071f40c960493d97dfe86a6dbb68b5fb2e26f6ee530a85cb9424a6ea7e
Process took too long(120.0 min).../nMoving on to next code
--
Decoding:  904d24eb73f3d3a4bfc0d2ac6386962b54688b75798cbfea28cfc699093bb1ce


In [ ]:
print(f"{len(codes)-len(cracked)} codes not cracked")

In [ ]:
#Write results to file
f = open('results.txt',"w")
for x in cracked:
        f.write(x+'\n')
for code in fin:
    f.write(code+": \n")
f.close()